# Model Training

## Import Library

In [2]:
import tensorflow as tf
import numpy as np 
import matplotlib.pyplot as plt
import os

## Train

In [3]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    min_lr=1e-8,
    verbose=1
)

In [4]:
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="Checkpoint/Softmaxede/TLe_{epoch:02d}.keras",
    save_best_only=True,
    mode = max,
    monitor='val_accuracy',
    verbose=1
)

/tmp/ipykernel_37345/3352014063.py:1: UserWarning: ModelCheckpoint mode '<built-in function max>' is unknown, fallback to auto mode.
  checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(


In [5]:
traingen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range = 20,
    width_shift_range= 0.2,
    height_shift_range= 0.2,
    brightness_range= [0.8, 1.2],
    shear_range= 0.2,
    zoom_range= [0.9,1.2],
    horizontal_flip = True,
    channel_shift_range=10,
    fill_mode= 'nearest',
)

valgen = tf.keras.preprocessing.image.ImageDataGenerator(
)

train_dataset = traingen.flow_from_directory(
    './Datajadi/train',
    target_size=(256,256),
    class_mode= 'categorical',
    batch_size= 16,
    shuffle=True
)


val_dataset = valgen.flow_from_directory(
    './Datajadi/val',
    target_size=(256,256),
    class_mode= 'categorical',
    batch_size= 16,
)


test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    './Datajadi/test',
    image_size=(256, 256),
    batch_size=16,
    label_mode='categorical'
)


Found 80800 images belonging to 101 classes.
Found 10100 images belonging to 101 classes.
Found 10100 files belonging to 101 classes.


I0000 00:00:1733823111.150824   37345 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5563 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060, pci bus id: 0000:08:00.0, compute capability: 8.9


In [6]:
Efficientnet = tf.keras.applications.EfficientNetV2S(
include_top=False,
weights='imagenet',
input_shape = (256,256,3)
)
for layer in Efficientnet.layers:
    layer.trainable= False
Efficientnet.summary()

Model: "efficientnetv2-s"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling           │ (None, 256, 256,  │          0 │ input_layer[0][0] │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 128, 128,  │        648 │ rescaling[0][0]   │
│                     │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 128, 128,  │         96 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 128, 128,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 128, 128,  │      5,184 │ stem_activation[… │
│ (Conv2D)            │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_bn  │ (None, 128, 128,  │         96 │ block1a_project_… │
│ (BatchNormalizatio… │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_ac… │ (None, 128, 128,  │          0 │ block1a_project_… │
│ (Activation)        │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_add (Add)   │ (None, 128, 128,  │          0 │ block1a_project_… │
│                     │ 24)               │            │ stem_activation[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1b_project_co… │ (None, 128, 128,  │      5,184 │ block1a_add[0][0] │
│ (Conv2D)            │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1b_project_bn  │ (None, 128, 128,  │         96 │ block1b_project_… │
│ (BatchNormalizatio… │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1b_project_ac… │ (None, 128, 128,  │          0 │ block1b_project_… │
│ (Activation)        │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1b_drop        │ (None, 128, 128,  │          0 │ block1b_project_… │
│ (Dropout)           │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1b_add (Add)   │ (None, 128, 128,  │          0 │ block1b_drop[0][… │
│                     │ 24)               │            │ block1a_add[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2a_expand_conv │ (None, 64, 64,    │     20,736 │ block1b_add[0][0] │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2a_expand_bn   │ (None, 64, 64,    │        384 │ block2a_expand_c… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2a_expand_act… │ (None, 64, 64,    │          0 │ block2a_expand_b

 Total params: 20,331,360 (77.56 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 20,331,360 (77.56 MB)

In [7]:
last_layer = Efficientnet.get_layer('block6o_add')
last_output = last_layer.output
x = tf.keras.layers.Conv2D(256, (3,3),activation ='relu')(last_output)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(256, activation='relu',kernel_regularizer=tf.keras.regularizers.L2(0.001))(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(512, activation='relu',kernel_regularizer=tf.keras.regularizers.L2(0.001))(x)
x = tf.keras.layers.Dropout(0.5)(x)
x_out = tf.keras.layers.Dense(101, activation='softmax')(x)
model = tf.keras.Model(Efficientnet.input, x_out)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model.fit(train_dataset, epochs=10, validation_data=val_dataset)

In [8]:
model.save('Softmmaaxx/Softmax_Epochs10.h5')

In [9]:
mulaidari = "block5a_expand_conv"
Kondisi = False
for layer in model.layers:
    if layer.name == mulaidari:
        Kondisi = True
    layer.trainable = Kondisi
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling           │ (None, 256, 256,  │          0 │ input_layer[0][0] │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 128, 128,  │        648 │ rescaling[0][0]   │
│                     │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 128, 128,  │         96 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 128, 128,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 128, 128,  │      5,184 │ stem_activation[… │
│ (Conv2D)            │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_bn  │ (None, 128, 128,  │         96 │ block1a_project_… │
│ (BatchNormalizatio… │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_ac… │ (None, 128, 128,  │          0 │ block1a_project_… │
│ (Activation)        │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_add (Add)   │ (None, 128, 128,  │          0 │ block1a_project_… │
│                     │ 24)               │            │ stem_activation[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1b_project_co… │ (None, 128, 128,  │      5,184 │ block1a_add[0][0] │
│ (Conv2D)            │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1b_project_bn  │ (None, 128, 128,  │         96 │ block1b_project_… │
│ (BatchNormalizatio… │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1b_project_ac… │ (None, 128, 128,  │          0 │ block1b_project_… │
│ (Activation)        │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1b_drop        │ (None, 128, 128,  │          0 │ block1b_project_… │
│ (Dropout)           │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1b_add (Add)   │ (None, 128, 128,  │          0 │ block1b_drop[0][… │
│                     │ 24)               │            │ block1a_add[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2a_expand_conv │ (None, 64, 64,    │     20,736 │ block1b_add[0][0] │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2a_expand_bn   │ (None, 64, 64,    │        384 │ block2a_expand_c… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2a_expand_act… │ (None, 64, 64,    │          0 │ block2a_expand_b

 Total params: 22,518,417 (85.90 MB)

 Trainable params: 18,865,453 (71.97 MB)

 Non-trainable params: 1,973,400 (7.53 MB)

 Optimizer params: 1,679,564 (6.41 MB)

In [10]:
model.fit(train_dataset, epochs=50, validation_data=val_dataset, callbacks=[reduce_lr],initial_epoch=10)

Epoch 11/50
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 769s 152ms/step - accuracy: 0.5707 - loss: 1.8737 - val_accuracy: 0.6898 - val_loss: 1.3648 - learning_rate: 1.0000e-04
Epoch 12/50
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 773s 153ms/step - accuracy: 0.5769 - loss: 1.8449 - val_accuracy: 0.6941 - val_loss: 1.3497 - learning_rate: 1.0000e-04
Epoch 13/50
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 772s 153ms/step - accuracy: 0.5844 - loss: 1.8056 - val_accuracy: 0.6945 - val_loss: 1.3340 - learning_rate: 1.0000e-04
Epoch 14/50
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 772s 153ms/step - accuracy: 0.5854 - loss: 1.7956 - val_accuracy: 0.6988 - val_loss: 1.3183 - learning_rate: 1.0000e-04
Epoch 15/50
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 777s 154ms/step - accuracy: 0.5899 - loss: 1.7742 - val_accuracy: 0.7053 - val_loss: 1.3004 - learning_rate: 1.0000e-04
Epoch 16/50
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 773s 153ms/step - accuracy: 0.5957 - loss: 1.7422 - val_accuracy: 0.7071 - val_loss: 1.2976 - learning_rate: 1.0000e-04
Epoch 17/50
5050/5050 

In [15]:
model.save('Softmmaaxx/Softmax_Epochs50.h5')

In [16]:
tf.keras.models.save_model(model, 'Softmmaaxx/SoftMax50Epochs.h5')

In [17]:
model.save_weights("Softmmaaxx/SoftyWeights_Epochs50.weights.h5")

In [14]:
val_loss, val_acc = model.evaluate(val_dataset)
tra_loss, tra_acc = model.evaluate(train_dataset)
test_loss, test_acc = model.evaluate(test_dataset)


print(f"Train Accuracy\t\t: {tra_acc * 100:.2f}% | Train Loss\t\t: {tra_loss}")
print(f"Validation Accuracy\t: {val_acc * 100:.2f}% | Validation Loss\t: {val_loss}")
print(f"Test Accuracy\t\t: {test_acc * 100:.2f}% | Test Loss \t\t: {test_loss}")

632/632 ━━━━━━━━━━━━━━━━━━━━ 16s 25ms/step - accuracy: 0.7365 - loss: 1.1358
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 783s 155ms/step - accuracy: 0.7740 - loss: 0.9561
632/632 ━━━━━━━━━━━━━━━━━━━━ 15s 24ms/step - accuracy: 0.7257 - loss: 1.1443
Train Accuracy		: 77.41% | Train Loss		: 0.9528151154518127
Validation Accuracy	: 74.27% | Validation Loss	: 1.1235365867614746
Test Accuracy		: 73.45% | Test Loss 		: 1.121769905090332


### Continue training

In [14]:
model = tf.keras.models.load_model('Softmmaaxx/Softmax_Epochs50.h5')

In [15]:
model.trainable = True

In [16]:
model.compile(optimizer=tf.keras.optimizers.AdamW(learning_rate=0.00005,weight_decay=0.004),loss='categorical_crossentropy',metrics=['accuracy'])

In [17]:
model.fit(train_dataset, epochs=10, validation_data=val_dataset, callbacks=[reduce_lr])

Epoch 1/10


E0000 00:00:1733807378.506584   21866 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1733807378.681110   21866 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1733807379.338994   21866 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1733807379.527090   21866 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1733807379.734631   21866 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:0

5050/5050 ━━━━━━━━━━━━━━━━━━━━ 1031s 190ms/step - accuracy: 0.5526 - loss: 1.9114 - val_accuracy: 0.7731 - val_loss: 0.9736 - learning_rate: 5.0000e-05
Epoch 2/10
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 853s 169ms/step - accuracy: 0.6912 - loss: 1.2902 - val_accuracy: 0.7952 - val_loss: 0.9007 - learning_rate: 5.0000e-05
Epoch 3/10
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 848s 168ms/step - accuracy: 0.7320 - loss: 1.1367 - val_accuracy: 0.8075 - val_loss: 0.8730 - learning_rate: 5.0000e-05
Epoch 4/10
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 850s 168ms/step - accuracy: 0.7603 - loss: 1.0220 - val_accuracy: 0.8227 - val_loss: 0.8201 - learning_rate: 5.0000e-05
Epoch 5/10
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 862s 171ms/step - accuracy: 0.7835 - loss: 0.9214 - val_accuracy: 0.8215 - val_loss: 0.8365 - learning_rate: 5.0000e-05
Epoch 6/10
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 881s 175ms/step - accuracy: 0.8000 - loss: 0.8511 - val_accuracy: 0.8274 - val_loss: 0.8086 - learning_rate: 5.0000e-05
Epoch 7/10
5050/5050 ━━━━━━━━━━━━━━━━━

In [18]:
model.save_weights("Softmmaaxx/SoftyWeights_Epochs60.weights.h5")

In [21]:
val_loss, val_acc = model.evaluate(val_dataset)
tra_loss, tra_acc = model.evaluate(train_dataset)
test_loss, test_acc = model.evaluate(test_dataset)


print(f"Train Accuracy\t\t: {tra_acc * 100:.2f}% | Train Loss\t\t: {tra_loss}")
print(f"Validation Accuracy\t: {val_acc * 100:.2f}% | Validation Loss\t: {val_loss}")
print(f"Test Accuracy\t\t: {test_acc * 100:.2f}% | Test Loss \t\t: {test_loss}")

632/632 ━━━━━━━━━━━━━━━━━━━━ 21s 32ms/step - accuracy: 0.8348 - loss: 0.8150
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 814s 161ms/step - accuracy: 0.9517 - loss: 0.2720
632/632 ━━━━━━━━━━━━━━━━━━━━ 15s 24ms/step - accuracy: 0.8317 - loss: 0.8451
Train Accuracy		: 95.14% | Train Loss		: 0.2718205153942108
Validation Accuracy	: 83.52% | Validation Loss	: 0.8248896598815918
Test Accuracy		: 83.06% | Test Loss 		: 0.8396679759025574


In [8]:
model.load_weights('Softmmaaxx/SoftyWeights_Epochs60.weights.h5')

/home/usle/Bangkit/myenv/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 646 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [10]:
model.save('Softmmaaxx/Softmax_Epochs60.h5')

In [11]:
model.save('Softmmaaxx/Softmax_Epochs60.keras')